In [20]:
import vrep 
import sys
import time 
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

from tank import *

In [21]:
vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

if clientID!=-1:
    print ("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

#create instance of Tank
tank=Tank(clientID)

Connected to remote API server


In [22]:
proximity_sensors=["EN","ES","NE","NW","SE","SW","WN","WS"]
proximity_sensors_handles=[0]*8

# get handle to proximity sensors
for i in range(len(proximity_sensors)):
    err_code,proximity_sensors_handles[i] = vrep.simxGetObjectHandle(clientID,"Proximity_sensor_"+proximity_sensors[i], vrep.simx_opmode_blocking)
    
#read and print values from proximity sensors
#first reading should be done with simx_opmode_streaming, further with simx_opmode_buffer parameter
for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_streaming)

In [23]:
# tank.forward(5)

# #continue reading and printing values from proximity sensors
# t = time.time()
# counter = 0
# while (time.time()-t)<15: # read values for 5 seconds
#     counter +=1

#     for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
#         err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
#         if(err_code == 0):
#             if counter % 5000 == 0:
#                 print("Proximity_sensor_"+sensor_name, np.linalg.norm(detectedPoint))
#                 print()
#     # print()
    
# tank.stop()

In [24]:
# tank.forward(5)


In [25]:
# tank.turn_right()

In [26]:

    # 
# tank.stop()

In [27]:
def search_for_para_place(search_time = 10):
    tank.forward(5)
    t = time.time()
    # condition = True
    values = []
    while True:
        
        condition = True
        if (time.time()-t)>search_time:
            tank.stop()
            print("Didn't found parking spot ")
            return 0
        important_list = ['EN',"ES","NE","SE"]
        values = []
        for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
            
            if sensor_name in important_list:
                
                err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
                if(err_code == 0):
                    values.append(np.linalg.norm(detectedPoint))
        if len(values) == 4:
            
            for value in values:
                if ((value<1.3)|(value>1.85)):   # if any condition not in range we will be out of range
                    condition = False

            # if we reached that point that means that all value are in defined range = we found parking spot
            
            if condition:
                tank.stop()
                print("Found parking spot")
                return 1 
    

                
                    
                        

In [28]:
# search_for_para_place()

In [29]:
# now after we found parking spot, we need to position out tank correctly.
# to do that we will focus on SE sensor that should represent distance from 'car behind'
# around 3 we want tank to stop 
def prepare_parking_logic():
    # for now let's ignore spped 
    
    # rules
    in_se = ctrl.Antecedent(np.arange(1.3,3.5,0.05),'in_se')
    out_velo = ctrl.Consequent(np.arange(0,5.1,0.1),'out_velo')
    out_velo.defuzzify_method = 'centroid'
    in_se['low'] = fuzz.trimf(in_se.universe,[1.3,1.3,2.2])
    in_se['medium'] = fuzz.trimf(in_se.universe,[1.3,2.2,3.5])
    in_se['high'] = fuzz.trimf(in_se.universe,[2.2,3.5,3.5])
    out_velo['low'] = fuzz.trimf(out_velo.universe,[0,0,2.1])

    out_velo['medium'] = fuzz.trimf(out_velo.universe,[0,2.1,2.5])

    out_velo['high'] = fuzz.trimf(out_velo.universe,[2.5,5,5])
    
    acc_rule = ctrl.Rule(antecedent= in_se['low'] ,consequent = out_velo['high'])

    rul0 = ctrl.Rule(antecedent=(in_se['medium'] ),consequent=out_velo['medium'])

    dec_rule = ctrl.Rule(antecedent=in_se['high'],consequent=out_velo['low'])
    speed_ctrl = ctrl.ControlSystem([dec_rule,acc_rule,rul0])
    velo_updater = ctrl.ControlSystemSimulation(speed_ctrl)
    
    
    
    
    return velo_updater

In [30]:
step1 = prepare_parking_logic()

In [31]:
def prepare_parking(step1 = step1):
    sensor = "SE"
    counter = 0
    keep_track_of_previous_value = 0
    while True:
        
        for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
            if sensor_name ==sensor:
                err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
                
                if(err_code == 0): 
                    distance_data = np.linalg.norm(detectedPoint)
                    if distance_data > keep_track_of_previous_value:  
                        # sensor czasami zaczyna zwracać irracjonalnie niskie wartości, pomimo że na symulacji widac, ze jeszcze nie łapie kolejnego auta więc spodziewamy się co raz większych odległości
                        counter +=1
                        step1.input['in_se'] = distance_data
                        step1.compute()
                        tank.forward(step1.output['out_velo'])
                        keep_track_of_previous_value = distance_data
                        if counter % 10 == 0:
                            print("Proximity_sensor",distance_data,"Velocity",tank.rightvelocity)
                        
        if tank.rightvelocity < 0.95:
            print("Velocity below 0.95 stoping the tank; Tank prepared to begin parking")
            tank.stop()
            return 
                        
                        
                    

In [32]:
def parking_combined():
    place = search_for_para_place()
    if place == 1:
        prepare_parking()
    else:
        print("Place not found")
        return 0 
    
    # time.sleep(10)
    # vrep.simxStopSimulation(clientID,vrep.simx_opmode_oneshot) # stop the simulation in vrep     
    

In [33]:
parking_combined()

Found parking spot
Proximity_sensor 1.4549874443996136 Velocity 3.4537317254517923
Proximity_sensor 1.558863273126428 Velocity 3.1531844918263423
Proximity_sensor 1.6726069130453525 Velocity 2.891870132651166
Proximity_sensor 1.7515535811681857 Velocity 2.7302270866100344
Proximity_sensor 1.8403344795729975 Velocity 2.553563641622142
Proximity_sensor 1.9159787534767703 Velocity 2.3978590144438483
Proximity_sensor 1.9972571500266059 Velocity 2.2136926883648527
Proximity_sensor 2.063090862512691 Velocity 2.0404897398182698
Proximity_sensor 2.1200000001575203 Velocity 1.8618872670887443
Proximity_sensor 2.2400667627822184 Velocity 1.530963077339244
Proximity_sensor 2.287217048817606 Velocity 1.5246935252931364
Proximity_sensor 2.346445761503524 Velocity 1.5108432809487615
Proximity_sensor 2.4777374979084654 Velocity 1.4589152020112197
Proximity_sensor 2.692421411711214 Velocity 1.32515640096551
Velocity below 0.95 stoping the tank; Tank prepared to begin parking


In [34]:
tank.backward(0)
tank.turn_left()

In [35]:
#continue reading and printing values from proximity sensors
t = time.time()
counter = 0
# while (time.time()-t)<11: # read values for 5 seconds
    # counter +=1

for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
    err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
    if(err_code == 0):
        if counter % 5000 == 0:
            print("Proximity_sensor_"+sensor_name, np.linalg.norm(detectedPoint))
            print()
# print()

Proximity_sensor_EN 1.5402030070617647

Proximity_sensor_ES 0.9054902737702285

Proximity_sensor_NE 0.9054902737702285

Proximity_sensor_NW 2.2009539006435843

Proximity_sensor_SE 0.848785614315442

Proximity_sensor_SW 2.6243837968573636

Proximity_sensor_WN 2.064509418645978

Proximity_sensor_WS 3.260095947710696



In [312]:
tank.turn_right()